# LocalUISGD Test


In [2]:
#imports
import numpy as np 
import csv
import re
import sys
import math
from numpy import linalg as L
import pandas as pd
import random
from sklearn.preprocessing import normalize
from numba import njit
from collections import defaultdict
import random
from data import ImplicitData
import pandas as pd
from BISGD import BISGD
from LocalUBISGD_normal import LocalUBISGD_normal
from LocalUBISGD_normal_03 import LocalUBISGD_normal_03
from LocalUBISGD_normal_05 import LocalUBISGD_normal_05
from LocalUBISGD_normal_08 import LocalUBISGD_normal_08
from collections import defaultdict
from eval_implicit import EvalPrequential
from datetime import datetime
import getopt
print("plz")


plz


## Functions


In [3]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

def sumvector(V,x):
    result = []
    for v in V:
        temp = v + x
        result.append(temp)
    return result    

def multvector(x,V):
    result = []
    for v in V:
        temp = v * x
        result.append(temp)
    return result    

def TransformVec(V):
  
    #new = normalize(V[:,np.newaxis], axis=0).ravel()
    for i in range(len(V)):
        V[i] = sigmoid(V[i])
    D = LA.norm(V)
    D=1/D
    new = D * V
  

    return new


## Get the data

In [4]:
sys.path.append('./')
configPath = 'config'
dataPath = 'ymusic_gte90_6kusers.tsv'


Data = []
data = pd.read_csv("ymusic_gte90_6kusers.tsv","\t")
data = data.head(n=46000)
print(data)
Data = data.values.tolist()



/home/mmelo/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


       user_id  item_id
0       378723   124036
1       378723   608145
2       378723    48098
3       378723    80338
4       378723   334716
...        ...      ...
45995   295010   506081
45996   382938    74722
45997   221033   158827
45998   221033   603920
45999   221033    26129

[46000 rows x 2 columns]


# Optimal param for BISGD and IBISGD

## Iterations

In [4]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[0],Data_Transpose[1])

iterations = [5,10,15,20,25,30]

for i in iterations:
    print("iter")
    print(i)
    num_clusters = 8
    cl_num_iterations = i
    cl_learn_rate = 0.1
    cl_regularization = 0.1
    num_factors = 200
    num_iter = 9
    learn_rate = 0.25
    regularization = 0.45
    model = LocalUBISGD_normal(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
        num_factors, num_iter, learn_rate, regularization, random_seed = 10)
    eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
    start_recommend = datetime.now()
    print('start time', start_recommend)

    results=eval.EvaluateTime(0,stream.size, 100)

    print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
    end_recommend = datetime.now()
    print('end time', end_recommend)

    tempo = end_recommend - start_recommend

    print('run time', tempo)
    print('')
    print('get tuple',np.mean(results['time_get_tuple']))
    print('recommend',np.mean(results['time_recommend']))
    print('eval_point',np.mean(results['time_eval_point']))
    print('update',np.mean(results['time_update']))    


iter
5
start time 2022-06-27 22:51:55.907507
npmean(resuls[Recall@20]) 0.18207775093345047
end time 2022-06-27 23:09:54.640041
run time 0:17:58.732534

get tuple 3.820782122404679e-06
recommend 0.2145510446153419
eval_point 2.0975336619007857e-05
update 0.0022025212308634883
iter
10
start time 2022-06-27 23:09:54.811331
npmean(resuls[Recall@20]) 0.1777583187390543
end time 2022-06-27 23:27:35.089242
run time 0:17:40.277911

get tuple 3.5620046698528786e-06
recommend 0.20874895695629553
eval_point 2.00798235924983e-05
update 0.002307994790699171
iter
15
start time 2022-06-27 23:27:35.279462
npmean(resuls[Recall@20]) 0.17482215994826472
end time 2022-06-27 23:46:49.744450
run time 0:19:14.464988

get tuple 3.718194754227348e-06
recommend 0.22300790501812662
eval_point 2.1616532181223603e-05
update 0.0025944882527641627
iter
20
start time 2022-06-27 23:46:49.955250
npmean(resuls[Recall@20]) 0.18601126028583803
end time 2022-06-28 00:05:11.791685
run time 0:18:21.836435

get tuple 3.686438

## Eta

In [5]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[0],Data_Transpose[1])


etas = [0.0001, 0.001, 0.01, 0.1]

for e in etas:
    print("eta")
    print(e)
    num_clusters = 8
    cl_num_iterations = 25
    cl_learn_rate = e
    cl_regularization = 0.1
    num_factors = 200
    num_iter = 9
    learn_rate = 0.25
    regularization = 0.45
    model = LocalUBISGD_normal(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
        num_factors, num_iter, learn_rate, regularization, random_seed = 10)
    eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
    start_recommend = datetime.now()
    print('start time', start_recommend)

    results=eval.EvaluateTime(0,stream.size, 100)

    print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
    end_recommend = datetime.now()
    print('end time', end_recommend)

    tempo = end_recommend - start_recommend

    print('run time', tempo)
    print('')
    print('get tuple',np.mean(results['time_get_tuple']))
    print('recommend',np.mean(results['time_recommend']))
    print('eval_point',np.mean(results['time_eval_point']))
    print('update',np.mean(results['time_update'])) 

eta
0.0001
start time 2022-06-28 00:41:59.924390
npmean(resuls[Recall@20]) 0.16236888111888112
end time 2022-06-28 01:00:27.886923
run time 0:18:27.962533

get tuple 3.611129263172979e-06
recommend 0.21265821382924394
eval_point 2.0450176475764987e-05
update 0.002918927011282548
eta
0.001
start time 2022-06-28 01:00:28.079213
npmean(resuls[Recall@20]) 0.17484662576687116
end time 2022-06-28 01:20:09.274323
run time 0:19:41.195110

get tuple 3.7648885146431303e-06
recommend 0.22893582870832774
eval_point 2.0644363868038033e-05
update 0.002951113965200341
eta
0.01
start time 2022-06-28 01:20:09.471411
npmean(resuls[Recall@20]) 0.18823529411764706
end time 2022-06-28 01:38:36.226751
run time 0:18:26.755340

get tuple 3.6548013272492784e-06
recommend 0.21550355936657972
eval_point 2.073939976496384e-05
update 0.002942079150158426
eta
0.1
start time 2022-06-28 01:38:36.415735
npmean(resuls[Recall@20]) 0.18073850140358455
end time 2022-06-28 01:55:44.612856
run time 0:17:08.197121

get tuple

## Lambda

In [6]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[0],Data_Transpose[1])

lambdas = [0,0.0001, 0.001, 0.01, 0.1]   

for l in lambdas:
    print("lambda")
    print(l)
    num_clusters = 8
    cl_num_iterations = 25
    cl_learn_rate = 0.1
    cl_regularization = l
    num_factors = 200
    num_iter = 9
    learn_rate = 0.25
    regularization = 0.45
    model = LocalUBISGD_normal(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
        num_factors, num_iter, learn_rate, regularization, random_seed = 10)
    eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
    start_recommend = datetime.now()
    print('start time', start_recommend)

    results=eval.EvaluateTime(0,stream.size, 100)

    print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
    end_recommend = datetime.now()
    print('end time', end_recommend)

    tempo = end_recommend - start_recommend

    print('run time', tempo)
    print('')
    print('get tuple',np.mean(results['time_get_tuple']))
    print('recommend',np.mean(results['time_recommend']))
    print('eval_point',np.mean(results['time_eval_point']))
    print('update',np.mean(results['time_update'])) 

lambda
0
start time 2022-06-28 01:55:44.931622
npmean(resuls[Recall@20]) 0.17493297587131368
end time 2022-06-28 02:12:17.676885
run time 0:16:32.745263

get tuple 3.40388650479524e-06
recommend 0.19266471370615204
eval_point 1.8459009847223386e-05
update 0.002822876531144847
lambda
0.0001
start time 2022-06-28 02:12:17.855301
npmean(resuls[Recall@20]) 0.18826827690317016
end time 2022-06-28 02:31:22.607535
run time 0:19:04.752234

get tuple 3.6746108013650645e-06
recommend 0.21864296270645664
eval_point 1.9669404234484093e-05
update 0.0028334592114324154
lambda
0.001
start time 2022-06-28 02:31:22.811522
npmean(resuls[Recall@20]) 0.17829623287671234
end time 2022-06-28 02:50:45.615605
run time 0:19:22.804083

get tuple 3.8080267284227454e-06
recommend 0.22004159687928956
eval_point 2.037269407755708e-05
update 0.002917121514030125
lambda
0.01
start time 2022-06-28 02:50:45.829212
npmean(resuls[Recall@20]) 0.168637073388356
end time 2022-06-28 03:09:21.043903
run time 0:18:35.214691

g

SyntaxError: invalid syntax (2083253283.py, line 1)

# With variance in the number of models

In [ ]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[0],Data_Transpose[1])

models = [5,10,15,20,25,30]   

for m in models:
    print("model")
    print(m)
    num_clusters = m
    cl_num_iterations = 25
    cl_learn_rate = 0.01
    cl_regularization = 0.0001
    num_factors = 200
    num_iter = 9
    learn_rate = 0.25
    regularization = 0.45
    model = LocalUBISGD_normal(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
        num_factors, num_iter, learn_rate, regularization, random_seed = 10)
    eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
    start_recommend = datetime.now()
    print('start time', start_recommend)

    results=eval.EvaluateTime(0,stream.size, 100)

    print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
    end_recommend = datetime.now()
    print('end time', end_recommend)

    tempo = end_recommend - start_recommend

    print('run time', tempo)
    print('')
    print('get tuple',np.mean(results['time_get_tuple']))
    print('recommend',np.mean(results['time_recommend']))
    print('eval_point',np.mean(results['time_eval_point']))
    print('update',np.mean(results['time_update'])) 

## Full data test

In [ ]:
#imports
import numpy as np 
import csv
import re
import sys
import math
from numpy import linalg as L
import pandas as pd
import random
from sklearn.preprocessing import normalize
from numba import njit
from collections import defaultdict
import random
from data import ImplicitData
import pandas as pd
from BISGD import BISGD
from LocalUBISGD_normal import LocalUBISGD_normal
from LocalUBISGD_normal_03 import LocalUBISGD_normal_03
from LocalUBISGD_normal_05 import LocalUBISGD_normal_05
from LocalUBISGD_normal_08 import LocalUBISGD_normal_08
from collections import defaultdict
from eval_implicit import EvalPrequential
from datetime import datetime
import getopt

sys.path.append('./')
configPath = 'config'
dataPath = 'ymusic_gte90_6kusers.tsv'


Data = []
data = pd.read_csv("ymusic_gte90_6kusers.tsv","\t")

Data = data.values.tolist()

Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[0],Data_Transpose[1])

num_clusters = 8
cl_num_iterations = 25
cl_learn_rate = 0.01
cl_regularization = 0.0001
num_factors = 200
num_iter = 9
learn_rate = 0.25
regularization = 0.45
model = LocalUBISGD_normal(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
    num_factors, num_iter, learn_rate, regularization, random_seed = 10)
eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
start_recommend = datetime.now()
print('start time', start_recommend)

results=eval.EvaluateTime(0,stream.size, 100)

print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
end_recommend = datetime.now()
print('end time', end_recommend)

tempo = end_recommend - start_recommend

print('run time', tempo)
print('')
print('get tuple',np.mean(results['time_get_tuple']))
print('recommend',np.mean(results['time_recommend']))
print('eval_point',np.mean(results['time_eval_point']))
print('update',np.mean(results['time_update'])) 

# Variation in overlap_factor

## 0.333333

In [ ]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[0],Data_Transpose[1])



num_clusters = 8
cl_num_iterations = 25
cl_learn_rate = 0.01
cl_regularization = 0.0001
num_factors = 200
num_iter = 9
learn_rate = 0.25
regularization = 0.45
model = LocalUBISGD_normal_03(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
    num_factors, num_iter, learn_rate, regularization, random_seed = 10)
eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
start_recommend = datetime.now()
print('start time', start_recommend)

results=eval.EvaluateTime(0,stream.size, 100)

print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
end_recommend = datetime.now()
print('end time', end_recommend)

tempo = end_recommend - start_recommend

print('run time', tempo)
print('')
print('get tuple',np.mean(results['time_get_tuple']))
print('recommend',np.mean(results['time_recommend']))
print('eval_point',np.mean(results['time_eval_point']))
print('update',np.mean(results['time_update'])) 

## 0.5

In [ ]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[0],Data_Transpose[1])



num_clusters = 8
cl_num_iterations = 20
cl_learn_rate =0.1
cl_regularization = 0.1
num_factors = 160
num_iter = 8
learn_rate = 0.1
regularization = 0.4
model = LocalUBISGD_normal_05(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
    num_factors, num_iter, learn_rate, regularization, random_seed = 10)
eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
start_recommend = datetime.now()
print('start time', start_recommend)

results=eval.EvaluateTime(0,stream.size, 100)

print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
end_recommend = datetime.now()
print('end time', end_recommend)

tempo = end_recommend - start_recommend

print('run time', tempo)
print('')
print('get tuple',np.mean(results['time_get_tuple']))
print('recommend',np.mean(results['time_recommend']))
print('eval_point',np.mean(results['time_eval_point']))
print('update',np.mean(results['time_update'])) 

## 0.8

In [ ]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[0],Data_Transpose[1])



num_clusters = 8
cl_num_iterations = 20
cl_learn_rate =0.1
cl_regularization = 0.1
num_factors = 160
num_iter = 8
learn_rate = 0.1
regularization = 0.4
model = LocalUBISGD_normal_08(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
    num_factors, num_iter, learn_rate, regularization, random_seed = 10)
eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
start_recommend = datetime.now()
print('start time', start_recommend)

results=eval.EvaluateTime(0,stream.size, 100)

print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
end_recommend = datetime.now()
print('end time', end_recommend)

tempo = end_recommend - start_recommend

print('run time', tempo)
print('')
print('get tuple',np.mean(results['time_get_tuple']))
print('recommend',np.mean(results['time_recommend']))
print('eval_point',np.mean(results['time_eval_point']))
print('update',np.mean(results['time_update'])) 

### ymusic_sample5_1

In [16]:
sys.path.append('./')
configPath = 'config'
dataPath = 'ymusic_gte90_6kusers.tsv'


Data = []
data = pd.read_csv("ymusic_sample5_1.csv")
print(data)
Data = data.values.tolist()

       Unnamed: 0  user_id  item_id
0            3873   795245    10843
1            3874   795245    56437
2            3875   795245    75757
3            3876   795245    96467
4            3877   795245   111214
...           ...      ...      ...
22775      476510   555020   475132
22776      476511   555020   141032
22777      476512   555020   201436
22778      476513   555020   125269
22779      476514   555020   141393

[22780 rows x 3 columns]


In [17]:

Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[1],Data_Transpose[2])

num_clusters = 8
cl_num_iterations = 25
cl_learn_rate = 0.01
cl_regularization = 0.0001
num_factors = 200
num_iter = 9
learn_rate = 0.25
regularization = 0.45
model = LocalUBISGD_normal(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
    num_factors, num_iter, learn_rate, regularization, random_seed = 10)
eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
start_recommend = datetime.now()
print('start time', start_recommend)

results=eval.EvaluateTime(0,stream.size, 100)

print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
end_recommend = datetime.now()
print('end time', end_recommend)

tempo = end_recommend - start_recommend

print('run time', tempo)
print('')
print('get tuple',np.mean(results['time_get_tuple']))
print('recommend',np.mean(results['time_recommend']))
print('eval_point',np.mean(results['time_eval_point']))
print('update',np.mean(results['time_update'])) 

start time 2022-09-20 19:04:19.746378
npmean(resuls[Recall@20]) 0.0811965811965812
end time 2022-09-20 19:06:00.576296
run time 0:01:40.829918

get tuple 2.601284850993839e-06
recommend 0.15607416324126414
eval_point 1.931699932130993e-05
update 0.0028170883707878775


### ymusic_sample5_2

In [18]:
sys.path.append('./')
configPath = 'config'
dataPath = 'ymusic_gte90_6kusers.tsv'


Data = []
data = pd.read_csv("ymusic_sample5_2.csv")
print(data)
Data = data.values.tolist()

       Unnamed: 0  user_id  item_id
0             785   218332    67176
1             786   218332    74262
2             787   218332   196852
3             788   218332   290893
4             789   218332   366723
...           ...      ...      ...
27915      476256   986546    53049
27916      476257   986546   381024
27917      476258   986546   392726
27918      476686   836553   309703
27919      476687   836553   219502

[27920 rows x 3 columns]


In [19]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[1],Data_Transpose[2])

num_clusters = 8
cl_num_iterations = 25
cl_learn_rate = 0.01
cl_regularization = 0.0001
num_factors = 200
num_iter = 9
learn_rate = 0.25
regularization = 0.45
model = LocalUBISGD_normal(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
    num_factors, num_iter, learn_rate, regularization, random_seed = 10)
eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
start_recommend = datetime.now()
print('start time', start_recommend)

results=eval.EvaluateTime(0,stream.size, 100)

print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
end_recommend = datetime.now()
print('end time', end_recommend)

tempo = end_recommend - start_recommend

print('run time', tempo)
print('')
print('get tuple',np.mean(results['time_get_tuple']))
print('recommend',np.mean(results['time_recommend']))
print('eval_point',np.mean(results['time_eval_point']))
print('update',np.mean(results['time_update'])) 

start time 2022-09-20 19:06:01.138333
npmean(resuls[Recall@20]) 0.08461538461538462
end time 2022-09-20 19:08:04.342010
run time 0:02:03.203677

get tuple 2.4811159232284415e-06
recommend 0.17938678356317372
eval_point 1.7482500809889574e-05
update 0.0027367414922632254


### ymusic_sample5_3

In [20]:
sys.path.append('./')
configPath = 'config'
dataPath = 'ymusic_gte90_6kusers.tsv'


Data = []
data = pd.read_csv("ymusic_sample5_3.csv")
print(data)
Data = data.values.tolist()

       Unnamed: 0  user_id  item_id
0            1792   292189    40476
1            1793   292189   110821
2            1794   292189   337957
3            1795   292189   392167
4            1796   292189   567378
...           ...      ...      ...
23055      476483   214741   137608
23056      476485   214741   551893
23057      476690   315275   385015
23058      476760   736333    59141
23059      476761   736333   343375

[23060 rows x 3 columns]


In [21]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[1],Data_Transpose[2])

num_clusters = 8
cl_num_iterations = 25
cl_learn_rate = 0.01
cl_regularization = 0.0001
num_factors = 200
num_iter = 9
learn_rate = 0.25
regularization = 0.45
model = LocalUBISGD_normal(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
    num_factors, num_iter, learn_rate, regularization, random_seed = 10)
eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
start_recommend = datetime.now()
print('start time', start_recommend)

results=eval.EvaluateTime(0,stream.size, 100)

print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
end_recommend = datetime.now()
print('end time', end_recommend)

tempo = end_recommend - start_recommend

print('run time', tempo)
print('')
print('get tuple',np.mean(results['time_get_tuple']))
print('recommend',np.mean(results['time_recommend']))
print('eval_point',np.mean(results['time_eval_point']))
print('update',np.mean(results['time_update'])) 

start time 2022-09-20 19:08:04.787556
npmean(resuls[Recall@20]) 0.1087866108786611
end time 2022-09-20 19:09:40.482302
run time 0:01:35.694746

get tuple 2.4513586231241614e-06
recommend 0.13790085425436746
eval_point 1.7526259482156283e-05
update 0.0027150409901958907


### ymusic_sample5_4

In [22]:
sys.path.append('./')
configPath = 'config'
dataPath = 'ymusic_gte90_6kusers.tsv'


Data = []
data = pd.read_csv("ymusic_sample5_4.csv")
print(data)
Data = data.values.tolist()

       Unnamed: 0  user_id  item_id
0             461   762863   388288
1             462   762863   556804
2             463   762863   539820
3             464   762863   369332
4             465   762863   384847
...           ...      ...      ...
29563      476878   791728    97526
29564      476879   791728   594258
29565      476880   791728   278513
29566      476881   791728   525766
29567      476882   791728   544937

[29568 rows x 3 columns]


In [23]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[1],Data_Transpose[2])

num_clusters = 8
cl_num_iterations = 25
cl_learn_rate = 0.01
cl_regularization = 0.0001
num_factors = 200
num_iter = 9
learn_rate = 0.25
regularization = 0.45
model = LocalUBISGD_normal(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
    num_factors, num_iter, learn_rate, regularization, random_seed = 10)
eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
start_recommend = datetime.now()
print('start time', start_recommend)

results=eval.EvaluateTime(0,stream.size, 100)

print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
end_recommend = datetime.now()
print('end time', end_recommend)

tempo = end_recommend - start_recommend

print('run time', tempo)
print('')
print('get tuple',np.mean(results['time_get_tuple']))
print('recommend',np.mean(results['time_recommend']))
print('eval_point',np.mean(results['time_eval_point']))
print('update',np.mean(results['time_update'])) 

start time 2022-09-20 19:09:41.171029
npmean(resuls[Recall@20]) 0.06134969325153374
end time 2022-09-20 19:12:04.324824
run time 0:02:23.153795

get tuple 2.4985731680155835e-06
recommend 0.18988314216122307
eval_point 1.6977450598968318e-05
update 0.0027424137136249833


### ymusic_sample5_5

In [24]:
sys.path.append('./')
configPath = 'config'
dataPath = 'ymusic_gte90_6kusers.tsv'


Data = []
data = pd.read_csv("ymusic_sample5_5.csv")
print(data)
Data = data.values.tolist()

       Unnamed: 0  user_id  item_id
0            1080   540444   196323
1            1081   540444   504528
2            1082   540444   314333
3            1083   540444   258823
4            1446   540444   196852
...           ...      ...      ...
27883      476395   214741   436267
27884      476481   214741    80180
27885      476482   214741    22329
27886      476483   214741   137608
27887      476485   214741   551893

[27888 rows x 3 columns]


In [25]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[1],Data_Transpose[2])

num_clusters = 8
cl_num_iterations = 25
cl_learn_rate = 0.01
cl_regularization = 0.0001
num_factors = 200
num_iter = 9
learn_rate = 0.25
regularization = 0.45
model = LocalUBISGD_normal(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
    num_factors, num_iter, learn_rate, regularization, random_seed = 10)
eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
start_recommend = datetime.now()
print('start time', start_recommend)

results=eval.EvaluateTime(0,stream.size, 100)

print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
end_recommend = datetime.now()
print('end time', end_recommend)

tempo = end_recommend - start_recommend

print('run time', tempo)
print('')
print('get tuple',np.mean(results['time_get_tuple']))
print('recommend',np.mean(results['time_recommend']))
print('eval_point',np.mean(results['time_eval_point']))
print('update',np.mean(results['time_update']))

start time 2022-09-20 19:12:05.119693
npmean(resuls[Recall@20]) 0.11971830985915492
end time 2022-09-20 19:14:25.016221
run time 0:02:19.896528

get tuple 2.535796343288542e-06
recommend 0.2256010502157077
eval_point 1.8652056304501816e-05
update 0.0027133378740432134
